In [1]:
from tensorflow import keras
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# 打开显示
tf.debugging.set_log_device_placement(True)

In [3]:
# 显示电脑上的物理GPU
gpus = tf.config.experimental.list_physical_devices('GPU')

In [4]:
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
# 需要在GPU没有初始化之前执行. 
tf.config.experimental.set_virtual_device_configuration(
    gpus[0],
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048),
    tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048),
    tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048)]
)

In [6]:
# 逻辑GPU
tf.config.experimental.list_logical_devices('GPU')

[LogicalDevice(name='/device:GPU:0', device_type='GPU'),
 LogicalDevice(name='/device:GPU:1', device_type='GPU'),
 LogicalDevice(name='/device:GPU:2', device_type='GPU')]

In [7]:
fashion_mnist = keras.datasets.fashion_mnist
(x_train_all, y_train_all), (x_test, y_test) = fashion_mnist.load_data()
x_valid, x_train = x_train_all[:5000], x_train_all[5000:]
y_valid, y_train = y_train_all[:5000], y_train_all[5000:]

print(x_valid.shape, y_valid.shape)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(5000, 28, 28) (5000,)
(55000, 28, 28) (55000,)
(10000, 28, 28) (10000,)


In [8]:
x_train.astype(np.float32).reshape(-1, 784).shape

(55000, 784)

In [9]:
# 标准化
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train.astype(np.float32).reshape(55000, -1))
x_valid_scaled = scaler.transform(x_valid.astype(np.float32).reshape(5000, -1))

x_test_scaled = scaler.transform(x_test.astype(np.float32).reshape(10000, -1))

In [10]:
def make_dataset(data, target, epochs, batch_size, shuffle=True):
    dataset = tf.data.Dataset.from_tensor_slices((data, target))
    if shuffle:
        dataset = dataset.shuffle(10000)
    dataset = dataset.repeat(epochs).batch(batch_size).prefetch(50)
    return dataset

batch_size = 64
epochs = 20
train_dataset = make_dataset(x_train_scaled, y_train, epochs, batch_size)

Executing op TensorSliceDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op DummySeedGenerator in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ShuffleDatasetV3 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RepeatDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op BatchDatasetV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:CPU:0


In [11]:
model = keras.models.Sequential()
model.add(keras.layers.Dense(512, activation='relu', input_shape=(784,)))
model.add(keras.layers.Dense(256, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Add in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                2570      
Total params: 535,818
Trainable params: 535,818
Non-trainable params: 0
_________________________________________________________________


In [13]:
eval_dataset = make_dataset(x_valid_scaled, y_valid, epochs=1, batch_size=32, shuffle=False)

Executing op TensorSliceDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RepeatDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op BatchDatasetV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:CPU:0


In [14]:
history = model.fit(train_dataset, 
         steps_per_epoch=x_train_scaled.shape[0] // batch_size,
         epochs=10,
         validation_data=eval_dataset)

Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ModelDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op OptimizeDatasetV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AnonymousIteratorV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MakeIterator in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op DatasetCardinality in device /job:localhost/replica:0/task:0/device:CPU:0
Epoch 1/10
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executin

Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
 31/859 [>.............................] - ETA: 7s - loss: 1.1859 - accuracy: 0.6031Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_

107/859 [==>...........................] - ETA: 4s - loss: 0.8438 - accuracy: 0.7062Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_

Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
190/859 [=====>........................] - ETA: 3s - loss: 0.7335 - accuracy: 0.7419Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_

Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
266/859 [========>.....................] - ETA: 2s - loss: 0.6787 - accuracy: 0.7599Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_

...

Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
694/859 [=======================>......] - ETA: 0s - loss: 0.1608 - accuracy: 0.9386Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_

Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing 

Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_590 in device /job:localhost/replica:0/task:0/device:GPU:0
855/859 [============================>.] - ETA: 0s - loss: 0.1599 - accuracy: 0.9391Executing op __inference_

Executing op __inference_test_function_2438 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_2438 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_2438 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_2438 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_2438 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_2438 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_2438 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_2438 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_2438 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_2438 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing 

Executing op __inference_test_function_2438 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_2438 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_2438 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_2438 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_2438 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_2438 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_2438 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_2438 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_2438 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_2438 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing 

In [15]:
model.evaluate(eval_dataset)

Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op DatasetCardinality in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op DatasetCardinality in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Equal in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op GreaterEqual in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ModelDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op OptimizeDatasetV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AnonymousIteratorV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MakeIterator in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ModelDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing

Executing op __inference_test_function_2438 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_2438 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_2438 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_2438 in device /job:localhost/replica:0/task:0/device:GPU:0
 66/157 [===========>..................] - ETA: 0s - loss: 0.3693 - accuracy: 0.8944Executing op __inference_test_function_2438 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_2438 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_2438 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_2438 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_2438 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_

Executing op __inference_test_function_2438 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_2438 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_2438 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_2438 in device /job:localhost/replica:0/task:0/device:GPU:0
145/157 [==========================>...] - ETA: 0s - loss: 0.3795 - accuracy: 0.8950Executing op __inference_test_function_2438 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_2438 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_2438 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_2438 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_2438 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_

[0.3731086552143097, 0.8956000208854675]